In [1]:
import os
import re
import importlib
import json
import torch
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from Utils.Prompt import zeroshot_react_agent_prompt
from typing import List, Dict, Any
from pandas import DataFrame
from transformers import pipeline


OPENAI_API_KEY = os.getenv('OPEN_AI_API')
actionMapping = {"AccommodationSearch":"accommodations", "RestaurantSearch":"restaurants", "AttractionSearch":"attraction","BusinessClusterSearch":"nearby","Planner":"planner"}

class ReactAgent:
    def __init__(self,
                 working_model,
                 react_llm_name,
                 planner_llm_name,
                 #add clustering agent model name here as well
                 mode: str = 'zero_shot',
                 tools: List[str] = None,
                 max_retries: int = 3,
                 ) -> None: 
        self.react_name = react_llm_name
        self.working_model = working_model
        self.answer = ''
        self.json_log = []
        self.mode = mode
        self.planner_name = planner_llm_name
        self.notebook = []
        self.max_retries = max_retries
        self.last_actions = []
        
        self.current_observation = ''
        self.current_data = None

        self.tools = self.load_tools(tools, planner_model_name=planner_llm_name)
        #print(self.tools)
        self.retry_record = {key: 0 for key in self.tools}
        #print(self.retry_record)
        self.retry_record['invalidAction'] = 0
        #print(self.retry_record)

        if self.mode == 'zero_shot':
            self.agent_prompt = zeroshot_react_agent_prompt

        if react_llm_name == 'meta-llama/Llama-3.1-8B':
            self.llm = pipeline(
                    "text-generation", model="meta-llama/Llama-3.1-8B", model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
                )

    def run(self,query, reset=True) -> None:
        self.query = query

        if reset:
            self.__reset_agent()

        while not self.is_finished():
            self.step()
        
        return self.answer, self.scratchpad, self.json_log

    
    def step(self) -> None:
        #record the log
        self.json_log.append({"step": self.step_n, "thought":"",
                              "action": "", "observation": "", "state":""})
        #thought
        self.scratchpad += f'\nThought {self.step_n}:'
        self.scratchpad += ' ' + self.prompt_agent() #
        print(self.scratchpad.split('\n')[-1])
        self.json_log[-1]['thought'] = self.scratchpad.split('\n')[-1].replace(f'\nThought {self.step_n}:',"")
        
        
        #act
        self.scratchpad += f'\nAction {self.step_n}:'
       
        #get the action prompted
        action = self.prompt_agent()
        ##

        if action == None or action == '' or action == '\n':
            self.scratchpad += " Your action is filtered due to content. Please make sure your action does not start with ['\\n', 'Thought', 'Action', 'Observation'] and assume all the actions are permitted in this environment and take the action again."
        else:
            self.scratchpad += ' ' + action

        if len(self.last_actions) > 0 and self.last_actions[-1] != action:
            self.last_actions.clear() 

        # refresh last_action list
        self.last_actions.append(action)

        self.json_log[-1]['action'] = self.scratchpad.split('\n')[-1].replace(f'\nAction {self.step_n}:',"")

        if len(self.last_actions) == 3:
            print("The same action has been repeated 3 times consecutively. So we stop here.")
            # self.log_file.write("The same action has been repeated 3 times consecutively. So we stop here.")
            self.json_log[-1]['state'] = 'same action 3 times repeated'
            self.finished = True
            return


        print(self.scratchpad.split('\n')[-1])
        
        
        #observation
        self.scratchpad += f'\nObservation {self.step_n}: '
        action_type, action_arg = parse_action(action)
        #print(action_type, action_arg)
        if action_type != "Planner":
            if action_type in actionMapping:
                pending_action = actionMapping[action_type]
            elif action_type not in actionMapping:
                pending_action = 'invalidAction'

            if pending_action in self.retry_record:
                if self.retry_record[pending_action] + 1 > self.max_retries:
                    action_type = 'Planner'
                    print(f"{pending_action} early stop due to {self.max_retries} max retries.")
                    self.json_log[-1]['state'] = f"{pending_action} early stop due to {self.max_retries} max retries."
                    self.finished = True
                    return # so if the max tries is reached, we stop the loop
            elif pending_action not in self.retry_record:
                if self.retry_record['invalidAction'] + 1 > self.max_retries:
                    action_type = 'Planner'
                    print(f"invalidAction Early stop due to {self.max_retries} max retries.")
                    # self.log_file.write(f"invalidAction early stop due to {self.max_retries} max retries.")
                    self.json_log[-1]['state'] = f"invalidAction early stop due to {self.max_retries} max retries."
                    self.finished = True
                    return

        if action_type == 'AccommodationSearch':
            try:
                if validate_accommodation_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['accommodations'].run(action_arg.split(',')[0],[p.strip() for p in action_arg.split('[')[1].strip('[]').split(',')])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += self.current_observation
                    self.notebook.append({'Description': 'Accommodation Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = 'Successful'
                    
            except ValueError as e:
                print(e)
                self.retry_record['accommodations'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['accommodations'] += 1
                self.current_observation = f'Illegal Accommodation Search. Please try again.'
                self.scratchpad += f'Illegal Accommodation Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'AttractionSearch':
            try:
                if validate_attraction_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['attractions'].run(action_arg.split(',')[0],[action_arg.split(',')[1].strip()[1:][:-1]])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += self.current_observation 
                    self.notebook.append({'Description': 'Attraction Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = f'Successful'
            except ValueError as e:
                print(e)
                self.retry_record['attractions'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['attractions'] += 1
                self.current_observation = f'Illegal Attraction Search. Please try again.'
                self.scratchpad += f'Illegal Attraction Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'RestaurantSearch': #action_arg = 'Cheap Budget, Indian, [Good Flavor, Good Value]'
            try:
                if validate_restaurant_parameters_format(action_arg):
                    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                    self.current_data = self.tools['restaurants'].run(action_arg.split('[')[0].split(',')[0].strip(),action_arg.split('[')[0].split(',')[1].strip(),[a.strip() for a in action_arg.split('[')[1].strip()[:-1].split(',')])
                    self.current_observation = str(to_string(self.current_data))
                    self.scratchpad += self.current_observation
                    self.notebook.append({'Description': 'Restaurant Choice', 'Content': self.current_data})
                    self.__reset_record()
                    self.json_log[-1]['state'] = f'Successful'
            except ValueError as e:
                print(e)
                self.retry_record['restaurants'] += 1
                self.current_observation = str(e)
                self.scratchpad += str(e)
                self.json_log[-1]['state'] = f'Illegal args. Parameter Error'
            except Exception as e:
                print(e)
                self.retry_record['restaurants'] += 1
                self.current_observation = f'Illegal Restaurant Search. Please try again.'
                self.scratchpad += f'Illegal Restaurant Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        elif action_type == 'BusinessClusterSearch': #action_arg = 'Cheap Budget, Indian, [Good Flavor, Good Value]'
            try:
                self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
                self.current_data = self.tools['nearby'].run(self.notebook)
                self.current_observation = str(to_string(self.current_data))
                self.scratchpad += self.current_observation
                self.notebook.append({'Description': 'Restaurant Choice', 'Content': self.current_data})
                self.__reset_record()
                self.json_log[-1]['state'] = f'Successful'
            
            except Exception as e:
                print(e)
                self.retry_record['restaurants'] += 1
                self.current_observation = f'Illegal Restaurant Search. Please try again.'
                self.scratchpad += f'Illegal Restaurant Search. Please try again.'
                self.json_log[-1]['state'] = f'Illegal args. Other Error'

        #elif action_type == 'NotebookWrite':
        #    self.scratchpad = self.scratchpad.replace(to_string(self.current_data).strip(),'Masked due to limited length. Make sure the data has been written in Notebook.')
        #    self.current_observation = str(self.tools['notebook'].write(self.current_data, action_arg))
        #    self.scratchpad  +=  self.current_observation
        #    self.json_log[-1]['state'] = f'Successful'

        elif action_type == 'Planner':
            #print(self.notebook)
            self.current_observation = str(self.tools['planner'].run(str(self.notebook),action_arg))
            self.scratchpad  +=  self.current_observation
            self.answer = self.current_observation
            self.json_log[-1]['state'] = f'Successful'
        else:
            self.retry_record['invalidAction'] += 1
            self.current_observation = 'Invalid Action. Valid Actions are AccommodationSearch[Budget,Preference] / AttractionSearch[Budget, Preference] / RestaurantSearch[Budget, Cuisine, Preference]/ Planner[Query].'
            self.scratchpad += self.current_observation
            self.json_log[-1]['state'] = f'invalidAction'
        
        #print(f'Observation {self.step_n}: ' + self.current_observation+'\n')
        # rite(f'Observation {self.step_n}: ' + self.current_observation+'\n')
        self.json_log[-1]['observation'] = self.current_observation
        self.step_n += 1

        if action_type and action_type == 'Planner':
            self.finished = True
            self.answer = self.current_observation

        #print(self.scratchpad)
            #print(self.json_log)
            #print(self.notebook) 
        return
    
    
    
    
    def is_finished(self) -> bool:
        return self.finished
    
    #def is_halted(self) -> bool:
    #    return ((self.step_n > self.max_steps) or (
    #                len(self.enc.encode(self._build_agent_prompt())) > self.max_token_length)) and not self.finished
    
    def __reset_agent(self) -> None:
        self.step_n = 1
        self.finished = False
        self.answer = ''
        self.scratchpad: str = ''
        self.json_log = []

    def prompt_agent(self) -> str:
        #print("we prompt the agents")
        while True:
            request = format_step(self.llm(self._build_agent_prompt(), max_new_tokens = 256, return_full_text=False, temperature = 0, do_sample=False)[0]['generated_text'].split('\n')[0])
            #print("here is the raw request: === ", request)
            return request  
        
    def __reset_record(self) -> None:
        self.retry_record = {key: 0 for key in self.retry_record}
        self.retry_record['invalidAction'] = 0

    def _build_agent_prompt(self) -> str:
        if self.mode == "zero_shot":
            return self.agent_prompt.format(
                query=self.query,
                scratchpad=self.scratchpad)
        
    def load_tools(self, tools: List[str], planner_model_name=None) -> Dict[str, Any]:
        tools_map = {}
        for tool_name in tools:
            module = importlib.import_module(f"tools.{tool_name}.apis") #
            
            if tool_name == 'planner' and planner_model_name is not None:
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])(model_name=planner_model_name)
            elif tool_name == 'nearby':
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])()
            else:
                tools_map[tool_name] = getattr(module, tool_name[0].upper()+tool_name[1:])(working_model = self.working_model)
        #print(tools_map)
        return tools_map
        

def format_step(step: str) -> str:
    return step.strip('\n').strip().replace('\n', '')

def parse_action(string):
    if ('BusinessClusterSearch' not in string):
        pattern = r'^(\w+)\[(.+)\]$'
        match = re.match(pattern, string)
        action_type = match.group(1)
        action_arg = match.group(2)
    else:
        action_type = 'BusinessClusterSearch'
        action_arg = ''
    return action_type,action_arg

#def parse action arg

def to_string(data) -> str:
    if data is not None:
        if type(data) == DataFrame:
            return data.to_string(index=False)
        else:
            return str(data)
    else:
        return str(None)
    
def validate_accommodation_parameters_format(action_arg):
    pattern = r"(.*\s*.*)\s*,\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, preference list.")
    budget = match.group(1).lower()
    preference_list = match.group(2)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget, and expensive budget. Please try again.")

    #preference
    preference = preference_list.split(',')
    preference_core = [p.lower().strip().split(' ')[-1].strip() for p in preference]
    preferenceInRange = True
    preferenceAccepted = ['location','service','safety','quality']
    for p in preference_core:
        if p not in preferenceAccepted:
            preferenceInRange = False

    if not preferenceInRange:
        raise ValueError("Wrong preference Input. Please try again.")
    return True
    
def validate_attraction_parameters_format(action_arg):
    pattern = r"(.*\s*.*)\s*,\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, Preference list.")
    budget = match.group(1).lower()
    preference_list = match.group(2)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget, and expensive budget. Please try again.")

    preference = preference_list.strip().split(',')
    if(len(preference) > 1 ):
        raise ValueError("Attraction only allows one preference. Please try again")
    if '-' in preference[0]:
        preference_core = preference[0].strip().split('-')[0].lower()
    else:
        preference_core = preference[0].strip().split(' ')[0].lower()
    preferenceAccepted = ["family","history","activity","nature","food","shopping"]
    preferenceIsInRange = False
    if(preference_core in preferenceAccepted):
        preferenceIsInRange = True
    if not preferenceIsInRange:
        raise ValueError("Preference parameter invalid. Only family oriented / history oriented / activity oriented / nature oriented / food oriented / and shopping oriented are allowed. Please try again.")
    return True

def validate_restaurant_parameters_format(action_arg):
    pattern = r"(.*\s*.*),\s*(.*),\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, cuisine, preference list.")
    budget = match.group(1).lower()
    cuisine = match.group(2).lower()
    preference_list = match.group(3)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    #print(budget)
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget and expensive budget. Please try again.")

    cuisine_accepted = ["us","mexican","irish","french","italian","greek","indian","chinese","japanese","korean","vietnamese","thai","asian fusion","middle eastern"]
    #print(cuisine)
    cuisineInRange = False
    if cuisine in cuisine_accepted:
        cuisineInRange = True
    if not cuisineInRange:
        raise ValueError("Cuisine not valid. Accepted cuisine is: US / Mexican / Irish / French / Italian / Greek / Indian / Chinese / Japanese / Korean / Vietnamese / Thai / Asian Fusion and Middle Eastern. Please try again.")

    preference_list = [p.lower().strip() for p in preference_list.split(',')]
    preference_core = [p.strip().split(' ')[-1] for p in preference_list]
    #print(preference_core)

    preferenceInRange = True
    preferenceAccepted = ['',"flavor","freshness","service","environment","value"]
    for p in preference_core:
        if p not in preferenceAccepted:
            preferenceInRange = False

    if not preferenceInRange:
        raise ValueError("Wrong preference Input. Accepted inputs are: good flavor / good freshness / good healthy/ good service / good environment / good value. Please try again.")    
    return True




if __name__ == "__main__":
    tools_list = ["attractions","accommodations","restaurants","nearby","planner"]
    modelList = ['gpt4o','gemini','llama318b']

    model = modelList[2]
    model_map = {'gpt4o': 'gpt-4o-2024-11-20','llama318b': 'meta-llama/Llama-3.1-8B'} 

    agent = ReactAgent(working_model = 'gpt4o', tools=tools_list, react_llm_name = model_map[model], planner_llm_name = model_map[model])
    toolUsePlans = []
    toolUseScratchpads = []
    toolUseLogs = []
    with open (f'Prompts/humanQuerys.jsonl', 'r') as file:
        humanquerys = [json.loads(line.strip()) for line in file]
    for i in range (20):
        query = humanquerys[i]['query']

        planner_results, scratchpad, action_log  = agent.run(query)

        toolUsePlans.append({"index": i+1, "plan": planner_results})
        toolUseScratchpads.append({"index": i+1, "scratchpad": scratchpad})
        toolUseLogs.append({"index": i+1, "log": action_log})
        
        with open (f'Output/{model}/plans/toolUsePlans.jsonl', 'w') as file:
            for plan in toolUsePlans:
                json.dump(plan, file)
                file.write('\n')
        with open (f'Output/{model}/plans/toolUseScratchpads.jsonl', 'w') as file:
            for scratchpad in toolUseScratchpads:
                json.dump(scratchpad, file)
                file.write('\n')
        with open (f'Output/{model}/plans/toolUseLogs.jsonl', 'w') as file:
            for log in toolUseLogs:
                json.dump(log, file)
                file.write('\n')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `

The same action has been repeated 3 times consecutively. So we stop here.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Wrong budget Input, valid ones include: cheap budget, moderate budget, and expensive budget. Please try again.


...

Wrong budget Input, valid ones include: cheap budget, moderate budget and expensive budget. Please try again.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The same action has been repeated 3 times consecutively. So we stop here.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWa

AttributeError: 'Planner' object has no attribute 'llm'